# Modelamiento no supervisado en base a tópicos

In [1]:
import sys
sys.path.insert(0, '..')
from utils.preprocesamiento import StemmerTokenizer

tokenizador = StemmerTokenizer(stem=False,rmv_punctuation=True)

In [2]:
from utils.cargar import df_caso
from utils.preprocesamiento import process_df

caso = 'laura'
df = df_caso(caso)

df = process_df(df,'comment','sel',verbose=True)

df = df.drop(columns=['user_id','team_id','gender','df','title','opt_left','opt_right','max_num','phase','time','curso'])

549 rows found with non string elements for column comment (11.03%)
Deleting 362 columns for which max target value is over 7 (7.27%)
4067 available rows after processing


In [3]:
from sklearn.model_selection import train_test_split

df_train, df_test, _, _ = train_test_split(df, df['sel'], test_size=.05, stratify=df['sel'], random_state=0)

In [4]:
tokenized_corpus = [tokenizador(document) for document in df_train['comment']]
tokenized_test = [tokenizador(document) for document in df_test['comment']]

In [5]:
import gensim
from gensim import corpora

# Create a dictionary from the tokenized corpus
dictionary = corpora.Dictionary(tokenized_corpus)

# Convert the tokenized corpus into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(doc) for doc in tokenized_corpus]

Usamos el número de tópicos con mejor métrica de coherencia

In [6]:
%%time
lda_model = gensim.models.LdaModel(doc_term_matrix, num_topics=3, id2word=dictionary, passes=10)

CPU times: user 12.1 s, sys: 56 µs, total: 12.1 s
Wall time: 12.1 s


In [7]:
# Print the generated topics
topics = lda_model.print_topics(num_topics=3)
for topic in topics:
    print(topic)

(0, '0.031*"error" + 0.019*"errores" + 0.014*"si" + 0.011*"aprender" + 0.011*"adecuado" + 0.010*"No" + 0.010*"haber" + 0.010*"trabajo" + 0.010*"renunciar" + 0.009*"mejor"')
(1, '0.033*"trabajo" + 0.032*"tiempo" + 0.026*"familia" + 0.021*"vida" + 0.016*"si" + 0.014*"Laura" + 0.013*"importante" + 0.012*"debe" + 0.010*"puede" + 0.009*"bien"')
(2, '0.050*"proyecto" + 0.023*"si" + 0.014*"renunciar" + 0.012*"impacto" + 0.010*"daño" + 0.010*"bien" + 0.009*"Si" + 0.008*"ambiente" + 0.007*"correcto" + 0.007*"quedarse"')


In [8]:
import pyLDAvis.gensim_models as gensimvis
import pyLDAvis

vis_data = gensimvis.prepare(lda_model, doc_term_matrix, dictionary)
pyLDAvis.display(vis_data)

In [9]:
print(' '.join(tokenized_test[0]) + '\n')

# Convert the tokenized document into a document-term matrix
doc_term_matrix = [dictionary.doc2bow(tokenized_test[0])]

# Get the topic probabilities for the new document
topic_probs = lda_model.get_document_topics(doc_term_matrix)[0]

# Print the topic probabilities
for topic, prob in topic_probs:
    print(f"Topic {topic}: {prob}")

Personalmente encuentro importante vida personal trabajo dado pierden momentos realmente importan Sin embargo debe existir mínimo dedicación existir

Topic 0: 0.01903020218014717
Topic 1: 0.905022382736206
Topic 2: 0.07594739645719528


## Correlaciones

In [10]:
from TM_utils import get_lda_embeddings
import pandas as pd

arr_test = get_lda_embeddings(lda_model,tokenizador,dictionary,df_test,'comment')
df_topics = pd.DataFrame(arr_test, columns=['Topic{}'.format(i+1) for i in range(len(topics))])

df_topics['sel'] = df_test['sel'].values

In [11]:
from scipy.stats import pearsonr

# Only compute pearson prod-moment correlations between feature
# columns and target column
target_col_name = 'sel'
feature_target_corr = {}
for col in df_topics:
    if target_col_name != col:
        feature_target_corr[col + '_' + target_col_name] = \
            pearsonr(df_topics[col], df_topics[target_col_name])[0]
print("Feature-Target Correlations")
print(feature_target_corr)

Feature-Target Correlations
{'Topic1_sel': 0.4588897047479901, 'Topic2_sel': -0.11364390466831782, 'Topic3_sel': -0.3248657430002373}
